In [3]:
exec(open('file_reader.py').read())

generating features..

 498293668655423488

 498486826269548545

 500280249629036544

 500298588992593920

 524923293711998976

 524936793633083394

 524941720249978880

 524948206023880704

 524961721744900097

 544274544174071809

 544294893146091520

 544315472075042818

 552788945017516032

 553480082996879360

 553553288625672192

 553561170637238272

 580352273001410560

 581153923987206146

 581290271997968384

 581359544682614784

 758159624122097664

 763098277986209792

 764927075522260992

 767725956706414592

 768859780240773121

 769988636754505729

 774991078265094144

 775057555865206784

 498293668655423488

 498486826269548545

 500280249629036544

 500298588992593920

 524923293711998976

 524936793633083394

 524941720249978880

 524948206023880704

 524961721744900097

 544274544174071809

 544294893146091520

 544315472075042818

 552788945017516032

 553480082996879360

 553553288625672192

 553561170637238272

 580352273001410560

 581153923987206146

 5812902719

In [4]:
df.head()


,classification,context_path,has_context,text,topic,opinion
580319078155468800,true,NaN,0,Germanwings Airbus A320 crashes in French Alps...,true,0
580319184652890113,false,NaN,0,BREAKING: 148 passengers were on board #German...,false,0
580320684305416192,unverified,NaN,0,Accident aircraft looks to be Germanwings (Air...,unverified,0
580321156508577792,false,NaN,0,Now hearing 148 passengers + crew on board the...,false,0
580322453928431617,false,NaN,0,German Wings airline tweeting now about report...,false,0


In [ ]:
import sys
sys.path.insert(1, "./feature-extraction/twitter-features")
from VulgarExtractor import VulgarExtractor
from EmbedExtractor import EmbedExtractor
ee = EmbedExtractor()
print("hi!")

In [ ]:
wordlist = VulgarExtractor.vulgarWords("badwords.txt") 
dftext = df[['text']]
result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
df['isVulgar'] = result

In [6]:
from TwitterParser import TwitterParser
textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
tagged_sents = TwitterParser.tag(textlist)
print(len(tagged_sents))
df['POS'] = tagged_sents
df.head()

NameError: name 'df' is not defined

In [5]:
import pickle

word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in df['POS']]
contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]
word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['POS']]

with open('feature-extraction/twitter-features/word_embedding_vectors_dev.pickle','rb') as we_pickle:
    word_em = pickle.load(we_pickle)
    
df['wordCount'] = word_counts
df['posCounts'] = pos_count_list
df['containsAdjective'] = contains_adjs
df['containsURL'] = contains_urls
df['containsEmoji'] = contains_emojis
df['containsAbbreviation'] = contains_abbrevs
df['wordEmbedding']=word_embeddings
print(df['wordEmbedding'])
df.head()

NameError: name 'df' is not defined

In [ ]:
for i, tag in enumerate(TwitterParser.tagset):
    tag_counts = []
    for pos_counts in df['posCounts']:
        tag_counts.append(pos_counts[i])
    column_name = 'num_' + tag
    df[column_name] = tag_counts
df.head()

In [110]:

df.loc[df.classification == 'true', 'classification'] = 1
df.loc[df.classification == 'false', 'classification'] = 0
df.loc[df.classification == 'unverified', 'classification'] = 2


attributes = []
# getting the labels
# You have to comment this out if you want only tweet ID to be in the features. 
# Note that by doing this, you will screw up the simple/tr,simple/dev test located after this
attributes = ['isVulgar', 'containsAdjective', 'containsURL', 'containsEmoji', 'containsAbbreviation']

dev_labels = df['classification']
dev_labels = [l for l in labels]
dev_labels = np.array(labels)

# getting the values as a list of lists
dev_values = df[attributes].values.tolist()


#Below puts the tweet ID as a feature. Comment this out if you aren't using tweetID
###for i,index in enumerate(df.index):
###    dev_values[i].append(int(index))


dev_values = np.array(dev_values)
dev_values

array([[False, False, False, False, False],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False,  True, False, False, False],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False,  True, False, False, False],
       [False,  True, False, False,  True],
       [False, False, False, False, False],
       [False, False, False, False,  True],
       [ True,  True, False, False, False],
       [False,  True, False, False, False],
       [False, False, False, False,  True],
       [False,  True, False, False, False],
       [False,  True, False, False, False],
       [False,  True, False, False, False],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False,  True, False, False, False],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False,  True, False, Fal

In [ ]:
import classifiers

# note predict_proba() gets probabilities for all 3 labels
#... and decision_tree_classifier uses decision_function() instead of predict_proba()... weird sklearn quirk
# NOTE: This is where you change the classifier type. Can pick from [naive_bayes, svm_classifier, decision_tree_classifier]
predictions, probabilities = classifiers.naive_bayes(dev_values, dev_labels, dev_values)

ps = []
for i, p in enumerate(predictions):
    if p == 0:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 2:
        ps.append('unverified')
    

# creates pairings of the prediction and the probability of the prediction
pred_probs_pairs = [[ps[i], probabilities[i][predictions[i]]] for i in range(len(predictions))]  

#now we make a dictionary of tweetID to the pred_probs_pairs
pred_dict = {index:pred_probs_pairs[i] for i,index in enumerate(df.index)}
pred_dict



In [112]:
with open('output/classifier_output/test.json', 'w') as outfile:
    json.dump(pred_dict, outfile)

In [113]:
## python3 scorer/score.py semeval2017-task8-dataset/traindev/rumoureval-subtaskB-dev.json output/classifier_output/test.json



## Training on simple/tr, testing on simple/dev

In [3]:
# Simply doing all of the transformations we did in the first few cells
import pandas as pd
import sys
import numpy as np
sys.path.insert(0, '/feature-extraction/twitter-features/')

tr_df = pd.read_pickle('output/simple/train_data_simple.pickle')

wordlist = VulgarExtractor.vulgarWords("badwords.txt") 
dftext = tr_df[['text']]
result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
tr_df['isVulgar'] = result

sys.path.insert(1, "./feature-extraction/twitter-features")
textlist = [txt.replace('\n','') for txt in tr_df['text'].tolist()]
tagged_sents = TwitterParser.tag(textlist)
tr_df['POS'] = tagged_sents

word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in tr_df['POS']]
pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in tr_df['POS']]
contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in tr_df['POS']]
contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in tr_df['POS']]
contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in tr_df['POS']]
contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in tr_df['POS']]

tr_df['wordCount'] = word_counts
tr_df['posCounts'] = pos_count_list
tr_df['containsAdjective'] = contains_adjs
tr_df['containsURL'] = contains_urls
tr_df['containsEmoji'] = contains_emojis
tr_df['containsAbbreviation'] = contains_abbrevs
print(tr_df['wordCount'])
for i, tag in enumerate(TwitterParser.tagset):
    tag_counts = []
    for pos_counts in tr_df['posCounts']:
        tag_counts.append(pos_counts[i])
    column_name = 'num_' + tag
    tr_df[column_name] = tag_counts

tr_df.head()

NameError: name 'VulgarExtractor' is not defined

In [10]:
# Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells

tr_df.loc[tr_df.classification == 'true', 'classification'] = 1
tr_df.loc[tr_df.classification == 'false', 'classification'] = 0
tr_df.loc[tr_df.classification == 'unverified', 'classification'] = 2
# getting the labels

attributes = ['isVulgar', 'containsAdjective', 'containsURL', 'containsEmoji', 'containsAbbreviation']


tr_labels = tr_df['classification']
tr_labels = [l for l in tr_labels]
tr_labels = np.array(tr_labels)


# getting the values as a list of lists
tr_values = tr_df[attributes].values.tolist()
tr_values = np.array(tr_values)


NameError: name 'np' is not defined

In [6]:
predictions, probabilities = classifiers.decision_tree_classifier(tr_values, tr_labels, dev_values)
print(probabilities)
ps = []
for i, p in enumerate(predictions):
    if p == 0:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 2:
        ps.append('unverified')
    

# creates pairings of the prediction and the probability of the prediction
pred_probs_pairs = [[ps[i], probabilities[i][predictions[i]]] for i in range(len(predictions))]  


pred_dict = {index:pred_probs_pairs[i] for i,index in enumerate(df.index)}


pred_dict


NameError: name 'classifiers' is not defined

In [7]:
with open('output/classifier_output/tr_test.json', 'w') as outfile:
    json.dump(pred_dict, outfile)
    
#test with:
### python3 scorer/score.py semeval2017-task8-dataset/traindev/rumoureval-subtaskB-dev.json output/classifier_output/tr_test.json

FileNotFoundError: [Errno 2] No such file or directory: 'output/classifier_output/tr_test.json'